In [2]:
"""
Retrain the YOLO model for your own dataset.
"""
#이미지 증강
import os
#from pathlib import Path

HOME_DIR = 'C:/upload/'

ANNO_DIR = 'C:/upload/'
#xml 파일들


print(ANNO_DIR)


files = os.listdir(ANNO_DIR)

print('파일 개수는:',len(files))

C:/upload/
파일 개수는: 4493


In [3]:
import numpy as np

import tensorflow.keras.backend as K
from keras.layers import Input, Lambda
from keras.models import *
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.python.ops import control_flow_ops

Using TensorFlow backend.


In [4]:
import sys
import os
#LOCAL_DIR = os.path.abspath("c:/test/SED")
#sys.path.append(LOCAL_DIR)

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

In [5]:
from train import get_classes, get_anchors
from train import create_model, data_generator, data_generator_wrapper

In [6]:
#학습을 위한 기반 환경 설정. 
annotation_path = 'c:/upload/annotation.csv'
log_dir = 'c:/test/logs/000/'  #epoch 당 weight 저장 장소로 사용할 될 폴더.
classes_path = 'c:/test/model_data/helmetvest_classes.txt'  #내가 클래스 2개로 설정.
anchors_path = 'c:/test/model_data/yolo_anchors.txt'
model_weights_path = 'c:/test/model_data/yolo.h5'


class_names = get_classes(classes_path)
num_classes = len(class_names) #클래스 개수
print("클래스 개수", num_classes)
anchors = get_anchors(anchors_path)
# anchors
# 검출객체 너비, 높이의 초기값으로 주어진 값들로 이 초기값들이 리사이즈되어 실제 검출 객체 크기가 된다.


print("-------------------CLASS NAMES-------------------")
print(class_names)
print("-------------------CLASS NAMES-------------------")


input_shape = (416,416)
#이미지 크기

클래스 개수 4
-------------------CLASS NAMES-------------------
['helmet', 'non_helmet', 'vest', 'non_vest']
-------------------CLASS NAMES-------------------


In [7]:
tiny = len(anchors)==6 # 기본값 세팅
# create_tiny_model(), create_model() 함수의 인자 설정을 원본 train.py에서 수정. 
if tiny:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path=model_weights_path)
else:
    # create_model 은 해당 패키지의 tarin.py 내부에 있는 클래스를 사용. 이 함수는 keras 모듈이 많이 사용한다.
    model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path=model_weights_path) 

# epoch 마다 call back 하여 모델 파일 저장.
# 이것 또한 Keras에서 많이 사용하는 checkpoint 저장 방식인듯 하다.
logging = TensorBoard(log_dir=log_dir)
# ModelCheckpoint는 모델이 학습하면서 정의한 조건을 만족했을 때 Model의 weight 값을 중간 저장
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    # period - 모델 저장 주기를 결정하는 것으로 값이 1인 경우 매 학습시마다 저장하게 됩니다. 
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

# ModelCheckpoint - 모델을 저장할 때 사용되는 콜백함수.
# ReduceLROnPlateau - 지정된 기간동안 평가지표에서 성능 향상이 일어나지 않으면 학습률을 조정하는 콜백
# save_best-only - True 인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장됨.
# save_weights_only - True인 경우, 모델의 weights만 저장됩니다

# monitor - 모델을 저장할 때, 기준이 되는 값을 지정함.
# validation set의 loss가 가장 작을 때 저장하고 싶으면 'val_loss'를 입력
# train set의 loss가 가장 작을 때 모델을 저장하고 싶으면 'loss'를 입력

# verbose - 1
# 1일 경우 Epoch의 상황과 loss 값이 보여짐. Epoch가 진행되는 막대도 보여줌.


# patience
#  - mode 파라미터에서 지정한 값으로 상태에 도달했을 때 바로 종료시키지 않고 몇 번 더 학습시킬 것인지 결정합니다.
#  monitor='val_loss' 이고 mode='min', patience=4이면 검증데이터셋의 비용함수가 최저점이 되어도
#  4번은 더 학습시켜라 라는 뜻

# factor - 학습률이 감소되는 요소

'''
Early Stopping.
  너무 많은 Epoch 은 overfitting 을 일으킨다.
  하지만 너무 적은 Epoch 은 underfitting 을 일으킨다.
  그래서 Epoch를 많이 돌리고 특정 시점에서 멈추도록 하여 성능이 더 이상 증가하지 않으면
  학습을 중지시키도록 함.
  
'''

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Create YOLOv3 model with 9 anchors and 4 classes.


C:\Users\user\anaconda3\envs\test\lib\site-packages\keras\engine\saving.py:1319: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 27) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
C:\Users\user\anaconda3\envs\test\lib\site-packages\keras\engine\saving.py:1319: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((27,) vs (255,)).
  weight_values[i].shape))
C:\Users\user\anaconda3\envs\test\lib\site-packages\keras\engine\saving.py:1319: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 27) vs (255, 512, 1, 1)).
  weight_values[i].shape))
C:\Users\user\anaconda3\envs\test\lib\site-packages\keras\engine\saving.py:1319: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((27,) vs (255,)).
  weight_values[i].shape))
C:\Users\user\anaconda3\envs\test\lib\site-packages\keras\engine\saving.py:1319: UserWarning: Skipping loa

Load weights c:/test/model_data/yolo.h5.
Freeze the first 249 layers of total 252 layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
val_split = 0.2  # train data : val_data 를 8 : 2 로 설정함.

with open(annotation_path) as f:
    # with_open 하면 annotation 파일이 txt이든 csv이든 상관없음.
    lines = f.readlines()

# 랜덤 시드 생성 및 lines 셔플하기
np.random.seed(10101)
#np.random.seed() : 난수 생성.
np.random.shuffle(lines)
np.random.seed(None)

# 데이터셋 나누기
valnum = int(len(lines)*val_split)
trainnum = len(lines) - valnum
print(trainnum)
print(valnum)

1797
449


In [9]:
'''
stable loss 얻기 위해서 frozen layers를 먼저 train 함.
weight 값은 처음에 random 값으로 초기화 되어 있는데 이 경우 최적화된 weight값을 찾으려면
많은 학습 횟수가 필요할 수 있다. 그런데 pretrained 모델에서 초기부터 이미 최적화된 
weight 값을 기반해서 학습을 하면 좋은 성능이 나타날 수 있다.
일단 처음에 weight값을 학습하지 않고 별도의 classification layer (fully connected layer)
부터 먼저 일정 수준 학습을 시킨 다음에 feature extractor를 포함한 전체를 학습
'''

'''
fully connected layer : 이미지를 정의된 라벨로 분류하는 데 사용
한층의 모든 뉴런이 다음층이 모든 뉴런과 연결된 상태로
2차원의 배열 형태 이미지를 1차원의 평탄화 작업을 통해 이미지를 분류하는데 사용되는 계층

feature extractor(특징 추출) : 컴퓨터가 스스로 학습을 할 때 컴퓨터가 입력받은 데이터를 분석해서 일정한
패턴이나 규칙을 찾아내려면 컴퓨터가 인지할 수 있는 데이터로 변환해주어야 함.
이 때 데이터 별로 어떤 특징을 가지고 있는지를 찾아내고, 그것을 토대로 데이터를 벡터로 변환하는 
작업을 말한다.

먼저 50epoch 로 classification layer를 학습.
classification layer를 학습 한 뒤에는 이제 feature map과 classification layer를 모두 포함한 전체 layer를 다시 50회 학습합니다. 
그래서 model.layers[i].trainable = True로 변경합니다.
'''

# feature map - 이미지가 가지는 edge, texture, 형태등을 표현
# classification layer - 
if True:
    # Model의 optimizer(최적화)와 loss, metrics를 선택
    #adam이 빠르고 성능 좋고 무난.
    model.compile(optimizer=Adam(lr=1e-3), loss={
        
        'yolo_loss': lambda y_true, y_pred: y_pred})

    batch_size = 8
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(trainnum, valnum, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:trainnum], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, trainnum//batch_size),
            # 훈련 샘플 수 / 배치 사이즈
            validation_data=data_generator_wrapper(lines[trainnum:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, valnum//batch_size),
            epochs=50,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')


# 결과가 좋지 않다면 train 이어서 진행. (두 학습이 모델을 이어서 함.)

if True:
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')
    # 모든 레이어 Unfreeze 한다. fine tuning 하기 위해서
    # 기존에 학습되어져 있는 모델을 기반으로 아키텍쳐를 새로운 목적(나의 이미지 데이터에 맞게)변형하고 이미 학습된 모델 
    # Weights로 부터 학습을 업데이트하는 방법

    batch_size = 8
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(trainnum, valnum, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:trainnum], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, trainnum//batch_size),
        validation_data=data_generator_wrapper(lines[trainnum:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, valnum//batch_size),
        epochs=100,
        initial_epoch=50,
        callbacks=[logging, checkpoint, lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

# data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes)
# 이 파라미터들을  data_generator()함
# -> 이미지 데이터와 박스 데이터 리시트를 만들고 어노테이션 라인들 섞어서 

# save_best_only - True, False
# True 인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장됨.
# 모델의 정확도가 최고값을 갱신했을 때만 저장하도록 하는 옵션


# save_weights_only - True, False
# Weights만 저장할 지 여부
# True인 경우, 모델의 weights만 저장됨.
# False인 경우, 모델 레이어 및 weights 모두 저장됨.
# layer 구성, 노드 갯수, 노드 정보, activation 정보 등의 값을 같이 저장한다

# fit_generator() - fit()은 전체 dataset을 한 번에 fit method로 통과시켜서 전체 dataset을 메모리에 로드할 수 있는, 
# 작은 크기의 dataset으로 학습을 시킬 때 사용.
# fit_generator()는 x,y 를 직접적으로 통과시키지 않고, generator를 통해 데이터를 불러오고 Multiprocessing을 진행할 때
# 데이터 중복을 막기 위해서 사용. 큰 크기의 dataset으로 학습을 시킬 때 사용.


Train on 1797 samples, val on 449 samples, with batch size 8.



Epoch 1/50
224/224 [==============================] - 846s 4s/step - loss: 472.8948 - val_loss: 82.6133

Epoch 2/50
224/224 [==============================] - 818s 4s/step - loss: 56.2895 - val_loss: 53.0884
Epoch 3/50
224/224 [==============================] - 817s 4s/step - loss: 42.9995 - val_loss: 43.8043
Epoch 4/50
224/224 [==============================] - 833s 4s/step - loss: 38.5441 - val_loss: 35.2510
Epoch 5/50
224/224 [==============================] - 969s 4s/step - loss: 35.9535 - val_loss: 32.9630
Epoch 6/50
224/224 [==============================] - 1042s 5s/step - loss: 34.0208 - val_loss: 37.6276
Epoch 7/50
224/224 [==============================] - 1047s 5s/step - loss: 33.1691 - val_loss: 40.5894
Epoch 8/50
224/224 [==============================] - 1044s 5s/step - loss: 32.2862 - val_loss: 37.3922
Epoch 9/50
224/224 [==============================] - 1014s 5s/step - loss: 31.6249 - val_loss: 42.3116
Ep

224/224 [==============================] - 2643s 12s/step - loss: 16.6338 - val_loss: 20.6594
Epoch 69/100
224/224 [==============================] - 2492s 11s/step - loss: 16.5910 - val_loss: 14.8854
Epoch 70/100
224/224 [==============================] - 2515s 11s/step - loss: 16.6456 - val_loss: 21.2572

Epoch 00070: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
Epoch 71/100
224/224 [==============================] - 2510s 11s/step - loss: 16.5406 - val_loss: 18.3884
Epoch 00071: early stopping


In [11]:
####################################################################################################################